# Inferência estatística com Python

In [ ]:
# Instalando a principal biblioteca Python para aplicações de probabilidade:
# !pip install scipy

# Caso tenha problemas de versão, instale as seguintes bibliotecas:
# !pip install numpy==1.19.3
# !pip install pandas==1.1.4

In [ ]:
import pandas as pd

# ler os dados
dados = pd.read_csv('../Database/marketing_data.csv')
dados['sucesso_bin'] = dados['sucesso'].apply(lambda x: 1 if x == 'sim' else 0) # Criando variável de Bernoulli para o sucesso.

print(f"Shape dos Dados: {dados.shape}")
dados.head(10)

## Atenção

Aqui, estamos fazendo um exercício hipotético! Sim, essas 40 mil linhas do dataset chamado "dados" também pode ser uma amostra. Tudo depende de como os dados foram gerados.

Mas, aqui, estamos apenas "supondo" que se trata de todo o conjunto da população de clientes. A partir disso, vamos brincar de gerar amnostras aleatórias, como acontece no dia-a-dia dos negócios de forma real.

*Lembrando também que os dados são fictícios e, principalmente, não se referem a clientes do Itaú.*

# Intervalos de Confiança

**Média**

**Função para implementar intervalo de confiança da média**

In [ ]:
from numpy import mean, std
from scipy.stats import norm

def ic_media(dados, nivel_cofianca: float = 0.95, verbose: bool = True) -> list:
    """
    Função que constroí um iontervalo de confiança a partir de dados amostrais e um nível de confiança.
    
    :param dados: conjunto de dados amostrais para o qual queremos um intervalo de confiança.
    :type dados: pandas series ou list.
    :param nivel_cofianca: nível de confiança para o intervalo.
    :type nivel_cofianca: float.
    :param verbose: se True irá apresentar os resultados, caso contrário apenas retorna o intervalo.
    :type verbose: bool.
    
    :return: lista comos limites inferior e superior, respectivamente, para o intervalo de confiança.
    :rtype: list.
    """
    N = len(dados)  # Tamanho da amostra.
    avg = mean(dados)   # Valor da média amostral.
    std_ = std(dados, ddof=1)  # Desvio padrão calculado a partir da amostra.
    
    z_value = norm.ppf(1 - (1 - nivel_cofianca) / 2)  # Valor crítico de refrência da distribuição normall.
    
    if verbose:
        print(f"Média amostral: {avg:.2f}")
        print(f"Desvio Padrão amostral: {std_:.2f}")
        
    lim_inf = avg - z_value * std_ / (N ** 0.5)  # Limite inferior do intervalo de confiança (95%).
    lim_sup = avg + z_value * std_ / (N ** 0.5)  # Limite superior do intervalo de confiança (95%).
    
    if verbose:
        print(f"Intervalo de Confiança: [{lim_inf:.2f}, {lim_sup:.2f}]")

    return [lim_inf, lim_sup]

Exemplo de uso

In [ ]:
from numpy.random import seed
seed(1)

N = 100 # Tamanho da amostra.
amostra = dados.sample(N)   # Coletando uma amostra da nossa população.

ic = ic_media(amostra['saldo'], nivel_cofianca=0.95)

# Proporção

#### Função para implementar intervalo de confiança da proporção

In [ ]:
from numpy import mean
from scipy.stats import norm

def ic_proporcao(dados, nivel_cofianca: float = 0.95, verbose: bool = True) -> list:
    """
    Função que constroí um iontervalo de confiança a partir de dados amostrais e um nível de confiança.
    
    :param dados: conjunto de dados amostrais para o qual queremos um intervalo de confiança.
    :type dados: pandas series ou list.
    :param nivel_cofianca: nível de confiança para o intervalo.
    :type nivel_cofianca: float.
    :param verbose: se True irá apresentar os resultados, caso contrário apenas retorna o intervalo.
    :type verbose: bool.
    
    :return: lista comos limites inferior e superior, respectivamente, para o intervalo de confiança.
    :rtype: list.
    """
    N = len(dados)  # Tamanho da amostra.
    avg = mean(dados)   # Valor da média amostral.
    
    z_value = norm.ppf(1 - (1 - nivel_cofianca) / 2)  # Valor crítico de refrência da distribuição normall.
    
    if verbose:
        print(f"Prporção amostral: {avg:.2f}")
        
    lim_inf = avg - z_value * (avg * (1 - avg) / N) ** 0.5  # Limite inferior do intervalo de confiança (95%).
    lim_sup = avg + z_value * (avg * (1 - avg) / N) ** 0.5  # Limite superior do intervalo de confiança (95%).
    
    if verbose:
        print(f"Intervalo de Confiança: [{lim_inf:.2f}, {lim_sup:.2f}]")
        
    return [lim_inf, lim_sup]

In [ ]:
import numpy as np
np.random.seed(1) # Fixando a aleatoriamente para garantie que tenhamos os mesmos resultados
from scipy.stats import norm

N = 100 # Tamanho da amostra.
amostra = dados.sample(N)   # Coletando uma amostra da nossa população.

ic = ic_proporcao(amostra['sucesso_bin'], nivel_cofianca=0.95)

# Teste de hipótese (uma subpopulação)

**Média**

Função para implementar teste de hipótese para a média

In [ ]:
from numpy import mean, std, sqrt
from scipy.stats import norm


def th_media(dados, h0, nivel_significancia: float = 0.05, verbose: bool = True) -> list:
    """
    Função que realiza um teste de hipóteses a partir de dados amostrais e um nível de significância.
    
    :param dados: conjunto de dados amostrais para o qual queremos testar uma hipótese sobre um estimador pontual.
    :type dados: pandas series ou list.
    :param h0: hipótese nula. Deve seguir o padrão: '< 14000', ou '> 14000', ou ainda '= 14000', substituindo 14000 pelo valor de refrência para a a sua aplicação.
    :type h0: str.
    :param nivel_cofianca: nível de significância a ser comparado com o p-valor.
    :type nivel_cofianca: float.
    :param verbose: se True irá apresentar os resultados, caso contrário apenas retorna o p-valor.
    :type verbose: bool.
    
    :return: p-valor, ou seja, a probabilidade calculada de cometermos um erro tipo I.
    :rtype: float.
    """
    N = len(dados)  # Tamanho da amostra.
    avg = mean(dados)   # Valor da média amostral.
    std_ = std(dados)  # Desvio padrão calculado a partir da amostra.
    h0_num = float(''.join([x for x in h0 if x in '0123456789']))
    
    if verbose:
        print(f"Média amostral: {avg:.2f}")
        print(f"Desvio Padrão amostral: {std_:.2f}")
        
        et = (avg - h0_num) / (std_ / sqrt(N))  # Estatística de teste.
        
        if ('<' not in h0) & ('>' not in h0):
            p_valor = (norm.cdf(et)) * 2 if et < 0 else (1 - norm.cdf(et)) * 2  # P-valor
        else:
            p_valor = norm.cdf(et) if et < 0 else 1 - norm.cdf(et)  # P-valor
    
        if verbose:
            print(f"Valor da estatística de teste: {et:.4f}")
            print(f"p-valor: {p_valor:.4f}")
            
            if p_valor < nivel_significancia:
                print(f"A um nível de significância de {nivel_significancia * 100}%, podemos rejeitar a hipótese nula de {h0}.")
            else:
                print(f"A um nível de significância de {nivel_significancia * 100}%, não podemos rejeitar a hipótese nula de {h0}.")
                
    return p_valor

**Exemplo de uso**

**Hipoótese:**

In [ ]:
from numpy.random import seed
seed(1995)  # Garantindo que sempre obteremos o mesmo resultado aleatório.

N = 100 # Tamanho da amostra.
sample = dados.sample(N)

p_valor = th_media(sample['saldo'], h0='<= 14000', nivel_significancia=0.05)

**Hipótese**

In [ ]:
from numpy.random import seed
seed(1995)  # Garantindo que sempre obteremos o mesmo resultado aleatório.

N = 100 # Tamanho da amostra.
sample = dados.sample(N)

p_valor = th_media(sample['saldo'], h0='>= 16000', nivel_significancia=0.05)

**Hipótese**

In [ ]:
from numpy.random import seed
seed(1995)  # Garantindo que sempre obteremos o mesmo resultado aleatório.

N = 100 # Tamanho da amostra.
sample = dados.sample(N)

p_valor = th_media(sample['saldo'], h0='= 13000', nivel_significancia=0.05)

# Proporção

**Função para implementar teste de hipótese da proporção**

In [ ]:
from numpy import mean, sqrt
from scipy.stats import norm


def th_proporcao(dados, h0, nivel_significancia: float = 0.05, verbose: bool = True) -> list:
    """
    Função que realiza um teste de hipóteses a partir de dados amostrais e um nível de significância.
    
    :param dados: conjunto de dados amostrais para o qual queremos testar uma hipótese sobre um estimador pontual.
    :type dados: pandas series ou list.
    :param h0: hipótese nula. Deve seguir o padrão: '< 14000', ou '> 14000', ou ainda '= 14000', substituindo 14000 pelo valor de refrência para a a sua aplicação.
    :type h0: str.
    :param nivel_cofianca: nível de significância a ser comparado com o p-valor.
    :type nivel_cofianca: float.
    :param verbose: se True irá apresentar os resultados, caso contrário apenas retorna o p-valor.
    :type verbose: bool.
    
    :return: p-valor, ou seja, a probabilidade calculada de cometermos um erro tipo I.
    :rtype: float.
    """
    N = len(dados)  # Tamanho da amostra.
    avg = mean(dados)   # Valor da média amostral.
    std_ = sqrt(avg * (1 - avg))  # Desvio padrão calculado a partir da amostra.
    h0_num = float(''.join([x for x in h0 if x in '0123456789']))
    
    if verbose:
        print(f"Média amostral: {avg:.2f}")
        print(f"Desvio Padrão amostral: {std_:.2f}")
        
    et = (avg - h0_num) / (std_ / sqrt(N))  # Estatística de teste.
    
    if ('<' not in h0) & ('>' not in h0):
        p_valor = (norm.cdf(et)) * 2 if et < 0 else (1 - norm.cdf(et)) * 2  # P-valor
    else:
        p_valor = norm.cdf(et) if et < 0 else 1 - norm.cdf(et)  # P-valor
        
    if verbose:
        print(f"Valor da estatística de teste: {et:.4f}")
        print(f"p-valor: {p_valor:.4f}")
        
        if p_valor < nivel_significancia:
            print(f"A um nível de significância de {nivel_significancia * 100}%, podemos rejeitar a hipótese nula de {h0}.")
        else:
            print(f"A um nível de significância de {nivel_significancia * 100}%, não podemos rejeitar a hipótese nula de {h0}.")

## Exemplo de uso

**Hipótese**

In [ ]:
from numpy.random import seed
seed(1995)  # Garantindo que sempre obteremos o mesmo resultado aleatório.

N = 100 # Tamanho da amostra.
sample = dados.sample(N)

p_valor = th_proporcao(sample[sample['tipo_contato']=='whatsapp']['sucesso_bin'], h0='<= 0.11', nivel_significancia=0.1)